# Comparison star generator

In [ ]:
from stellarphot.gui_tools.comparison_functions import ComparisonViewer

## Change the settings in this cell as needed

In [ ]:
bright_mag_limit = 9
dim_mag_limit = 16
Cmag = 10

# 👉 File to save photometry apertures in
apertures_file = "aperture_locations.ecsv"

In [ ]:
comp_viewer = ComparisonViewer(
    directory="..",
    target_mag=Cmag,
    bright_mag_limit=bright_mag_limit,
    dim_mag_limit=dim_mag_limit,
    targets_from_file=None,
    photom_apertures_file=apertures_file,
    overwrite_outputs=True,
)
comp_viewer.box